# Build regression model

Please note that before running, make sure you are logged in by running `dm login EMAIL` in the terminal.

In [ ]:
!pip install deepmirror
# !dm login <YOUREMAIL>

In [ ]:
import time
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import r2_score
from tqdm import tqdm

from deepmirror.api import model_info, model_metadata, predict, train

In [ ]:
root = Path().cwd().parent
data_path_train = root / "data" / "data-reg.csv"
data_path_test = root / "data" / "data-reg-test.csv"

df_train = pd.read_csv(data_path_train)
df_test = pd.read_csv(data_path_test)

In [ ]:
df_train.head()

In [ ]:
model = train(
    model_name="test-reg-mode2123l",
    csv_file=data_path_train,
    smiles_column="SMILES",
    value_column="y",
)

In [ ]:
model

In [ ]:
model = model_info(model["model_id"])
with tqdm(desc="Training model", unit="sec") as pbar:
    while model["status"] != "completed":
        time.sleep(1)
        model = model_info(model["model_id"])
        pbar.set_postfix_str(f"Status: {model['status']}")
        pbar.update(1)

In [ ]:
model_metadata(model["model_id"])

In [ ]:
df_test

In [ ]:
predictions = predict(model["model_name"], smiles=df_test["SMILES"].tolist())

In [ ]:
y_pred = predictions["prediction"]
y_true = df_test["y"]

r2 = r2_score(y_true, y_pred)
print(f"R² Score: {r2:.4f}")

# Plot predicted vs actual values
plt.figure(figsize=(8, 6))
plt.scatter(y_true, y_pred, color="blue", alpha=0.5)

# Add perfect prediction line
min_val = min(min(y_true), min(y_pred))
max_val = max(max(y_true), max(y_pred))
plt.plot([min_val, max_val], [min_val, max_val], "r--")

plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)
plt.xlabel("Actual Values")
plt.ylabel("Predicted Values")
plt.title("Predicted vs Actual Values")
plt.text(0.05, 0.95, f"R² = {r2:.4f}", transform=plt.gca().transAxes)
plt.grid(True)
plt.show()